## Setup environment

In [1]:

from monai.utils import first, set_determinism
import torch
import matplotlib.pyplot as plt
import os
import torch.nn as nn
from data_preparation2 import DataHandling 
from datetime import datetime
import json
import numpy as np


## Set dataset path

In [2]:
config_file = 'config.json'

with open(config_file, 'r') as f:
    config = json.load(f)

ga_data_dir = config["ga_data_dir"]
fdg_data_dir = config["fdg_data_dir"]
log_dir = config["log_dir"]
output_dir = config["output_dir"]


In [2]:
# import os
# import glob

# ga_data_dir = '/homes/zshahpouri/DLP/Practic/OUT/'
# output_dir = '/homes/zshahpouri/DLP/ASC-PET-001/MAC'
# hint = 'DLdyn3'

# # List all DL processed .nii.gz files
# dl_files = glob.glob(os.path.join(output_dir, f'**/*{hint}*.nii.gz'), recursive=True)

# # Function to extract the common name from the filename
# def extract_common_name(filename):
#     # This removes the specific hint and file extension to find the common base name
#     return os.path.basename(filename).replace(f'_{hint}', '').split('.')[0]

# # Initialize an empty list to hold dictionaries
# test_dict_list = []

# for dl_path in dl_files:
#     common_name = extract_common_name(dl_path)
#     # Search for the corresponding original file
#     search_pattern = os.path.join(ga_data_dir, f'{common_name}*.nii.gz')
#     found_org_files = glob.glob(search_pattern)
    
#     if found_org_files:
#         # Each pair is stored in a separate dictionary
#         pair_dict = {
#             'predicted': dl_path,  # DL processed file
#             'reference': found_org_files[0]  # Assuming the first match is what you want
#         }
#         test_dict_list.append(pair_dict)

# # Print or use the list of dictionaries as needed
# for pair in test_dict_list:
#     print(f"Reference: {pair['reference']}, Predicted: {pair['predicted']}")


Reference: /homes/zshahpouri/DLP/ASC-PET-001/MAC/004240_C4_SH_.nii.gz, Predicted: /homes/zshahpouri/DLP/Practic/OUT/004240_C4_SH_/004240_C4_SH__DLdyn3.nii.gz
Reference: /homes/zshahpouri/DLP/ASC-PET-001/MAC/004256_C4_SH_.nii.gz, Predicted: /homes/zshahpouri/DLP/Practic/OUT/004256_C4_SH_/004256_C4_SH__DLdyn3.nii.gz
Reference: /homes/zshahpouri/DLP/ASC-PET-001/MAC/004277_C4_SH_.nii.gz, Predicted: /homes/zshahpouri/DLP/Practic/OUT/004277_C4_SH_/004277_C4_SH__DLdyn3.nii.gz
Reference: /homes/zshahpouri/DLP/ASC-PET-001/MAC/007038_C5_RJ_.nii.gz, Predicted: /homes/zshahpouri/DLP/Practic/OUT/007038_C5_RJ_/007038_C5_RJ__DLdyn3.nii.gz
Reference: /homes/zshahpouri/DLP/ASC-PET-001/MAC/007035_C5_RJ_.nii.gz, Predicted: /homes/zshahpouri/DLP/Practic/OUT/007035_C5_RJ_/007035_C5_RJ__DLdyn3.nii.gz
Reference: /homes/zshahpouri/DLP/ASC-PET-001/MAC/007042_C5_RJ_.nii.gz, Predicted: /homes/zshahpouri/DLP/Practic/OUT/007042_C5_RJ_/007042_C5_RJ__DLdyn3.nii.gz
Reference: /homes/zshahpouri/DLP/ASC-PET-001/MAC/007

In [3]:
import os
import glob
from utils import PairFinder
hint = 'dl_dyn2'
pair_finder = PairFinder(f'{ga_data_dir}/MAC', output_dir, hint)
test_dict_list = pair_finder.find_file_pairs()
test_dict_list

[{'predicted': '/students/2023-2024/master/Shahpouri/OUTPUT/007046_C5_RJ_/007046_C5_RJ__dl_dyn2.nii.gz',
  'reference': '/students/2023-2024/master/Shahpouri/DATA/Ga-PET/MAC/007046_C5_RJ_.nii.gz'},
 {'predicted': '/students/2023-2024/master/Shahpouri/OUTPUT/003232_C3_KH_/003232_C3_KH__dl_dyn2.nii.gz',
  'reference': '/students/2023-2024/master/Shahpouri/DATA/Ga-PET/MAC/003232_C3_KH_.nii.gz'},
 {'predicted': '/students/2023-2024/master/Shahpouri/OUTPUT/007067_C5_RJ_/007067_C5_RJ__dl_dyn2.nii.gz',
  'reference': '/students/2023-2024/master/Shahpouri/DATA/Ga-PET/MAC/007067_C5_RJ_.nii.gz'},
 {'predicted': '/students/2023-2024/master/Shahpouri/OUTPUT/004136_C4_SH_/004136_C4_SH__dl_dyn2.nii.gz',
  'reference': '/students/2023-2024/master/Shahpouri/DATA/Ga-PET/MAC/004136_C4_SH_.nii.gz'},
 {'predicted': '/students/2023-2024/master/Shahpouri/OUTPUT/001216_C1_RZ_/001216_C1_RZ__dl_dyn2.nii.gz',
  'reference': '/students/2023-2024/master/Shahpouri/DATA/Ga-PET/MAC/001216_C1_RZ_.nii.gz'},
 {'predict

In [26]:
test_dict_list = pair_finder.find_file_pairs()
test_dict_list

------------------
# Quantification Evaluation

In [4]:
import numpy as np
import torch
from skimage.metrics import structural_similarity as ssim
from math import log10, sqrt

def mean_error(predicted, reference):
    return np.mean(predicted - reference)

def mean_absolute_error(predicted, reference):
    return np.mean(np.abs(predicted - reference))

def relative_error(predicted, reference, epsilon=0.3):
    return np.mean((predicted - reference) / (reference + epsilon)) * 100

def absolute_relative_error(predicted, reference, epsilon=0.3):
    return np.mean(np.abs(predicted - reference) / (reference + epsilon)) * 100

def rmse(predicted, reference):
    return sqrt(np.mean((predicted - reference) ** 2))

def psnr(predicted, reference, peak):
    mse = np.mean((predicted - reference) ** 2)
    return 20 * log10(peak / sqrt(mse))

def calculate_ssim(predicted, reference):
    return ssim(predicted, reference, data_range=reference.max() - reference.min())


In [5]:
import nibabel as nib
import numpy as np
from skimage.metrics import structural_similarity as ssim
from math import log10, sqrt

def load_nifti_image(path):
    """Load a NIfTI image and return its data as a NumPy array."""
    return nib.load(path).get_fdata()

def calculate_metrics_for_pair(predicted_path, reference_path, scaling_factor=5):
    """
    Calculate metrics for a single pair of images, applying a scaling factor to the images.
    """
    predicted_img = load_nifti_image(predicted_path) * scaling_factor
    reference_img = load_nifti_image(reference_path) * scaling_factor
    

    peak = np.max([predicted_img.max(), reference_img.max()])
    metrics = {
        "mean_error": mean_error(predicted_img, reference_img),
        "mean_absolute_error": mean_absolute_error(predicted_img, reference_img),
        "relative_error": relative_error(predicted_img, reference_img),
        "absolute_relative_error": absolute_relative_error(predicted_img, reference_img),
        "rmse": rmse(predicted_img, reference_img),
        "psnr": psnr(predicted_img, reference_img, peak),
        "ssim": calculate_ssim(predicted_img, reference_img)
    }
    return metrics

def aggregate_metrics(metrics_list):
    """Aggregate metrics across all pairs and calculate mean and standard deviation."""
    aggregated_metrics = {key: [] for key in metrics_list[0]}
    for metrics in metrics_list:
        for key, value in metrics.items():
            aggregated_metrics[key].append(value)
    
    return {metric: (np.mean(values), np.std(values)) for metric, values in aggregated_metrics.items()}

# Calculate metrics for each pair and aggregate results
all_metrics = [calculate_metrics_for_pair(pair['predicted'], pair['reference']) for pair in test_dict_list]
metric_means_sds = aggregate_metrics(all_metrics)

# Print aggregated metrics
for metric, (mean, sd) in metric_means_sds.items():
    print(f"{metric}: {mean:.2f} ± {sd:.4f}")


mean_error: -0.13 ± 0.1053
mean_absolute_error: 0.29 ± 0.1097
relative_error: 11.28 ± 9.1598
absolute_relative_error: 24.58 ± 7.9870
rmse: 1.59 ± 0.9717
psnr: 43.46 ± 4.1015
ssim: 0.98 ± 0.0063


# Results for Dynunet without matching the networks prameters:
mean_error: -0.25 ± 0.0722
mean_absolute_error: 0.43 ± 0.0803
relative_error: 7.56 ± 7.7195
absolute_relative_error: 38.52 ± 4.3638
rmse: 2.10 ± 0.9815
psnr: 39.60 ± 3.0782
ssim: 0.97 ± 0.0040


# Results for Dyunet with matching the networks prameters:

mean_error: -0.27 ± 0.0802
mean_absolute_error: 0.37 ± 0.0524
relative_error: -14.82 ± 2.6155
absolute_relative_error: 35.17 ± 3.3576
rmse: 1.60 ± 0.3624
psnr: 41.57 ± 2.2030
ssim: 0.97 ± 0.0056